In [1]:
%cd /kaggle/input/fcosss/FCOS
!ls

/kaggle/input/fcosss/FCOS
README.md  coco_eval.py  demo.py  model
assets	   dataloader	 eval.py  train_voc.py


In [2]:
from model.fcos import FCOSDetector
import torch
from dataloader.VOC_dataset import VOCDataset
import math, time
from torch.utils.tensorboard import SummaryWriter

train_dataset=VOCDataset("/kaggle/input/pascal-voc-2012/VOC2012",resize_size=[512,800],split='train')
val_dataset=VOCDataset("/kaggle/input/pascal-voc-2012/VOC2012",resize_size=[512,800],split='val')

model=FCOSDetector(mode="training").cuda()

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

INFO=====>voc dataset init finished  ! !
INFO=====>voc dataset init finished  ! !


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 248MB/s]


In [7]:
sum(p.numel() for p in model.fcos_body.head.parameters() if p.requires_grad)

4920666

In [ ]:
from model.fcos import FCOSDetector
import torch
from dataloader.VOC_dataset import VOCDataset
import math, time
from torch.utils.tensorboard import SummaryWriter

train_dataset=VOCDataset("/kaggle/input/pascal-voc-2012/VOC2012",resize_size=[512,800],split='train')
val_dataset=VOCDataset("/kaggle/input/pascal-voc-2012/VOC2012",resize_size=[512,800],split='val')

model=FCOSDetector(mode="training").cuda()
# model.load_state_dict(torch.load("./checkpoints/voc_512x800_loss2.0635.pth"))
optimizer=torch.optim.Adam(model.parameters(),lr=1e-4)

BATCH_SIZE=8
EPOCHS=12
WARMPUP_STEPS_RATIO=0.12
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True,collate_fn=train_dataset.collate_fn)
val_loader=torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE,shuffle=True,collate_fn=val_dataset.collate_fn)
steps_per_epoch=len(train_dataset)//BATCH_SIZE
TOTAL_STEPS=steps_per_epoch*EPOCHS
WARMPUP_STEPS=TOTAL_STEPS*WARMPUP_STEPS_RATIO

GLOBAL_STEPS=1
LR_INIT=1e-4
LR_END=1e-5

writer=SummaryWriter(log_dir="/kaggle/working/logs")

def lr_func():
    if GLOBAL_STEPS<WARMPUP_STEPS:
        lr=GLOBAL_STEPS/WARMPUP_STEPS*LR_INIT
    else:
        lr=LR_END+0.5*(LR_INIT-LR_END)*(
            (1+math.cos((GLOBAL_STEPS-WARMPUP_STEPS)/(TOTAL_STEPS-WARMPUP_STEPS)*math.pi))
        )
    return float(lr)

model.train()

for epoch in range(EPOCHS):
    for epoch_step,data in enumerate(train_loader):

        batch_imgs,batch_boxes,batch_classes=data
        batch_imgs=batch_imgs.cuda()
        batch_boxes=batch_boxes.cuda()
        batch_classes=batch_classes.cuda()

        lr=lr_func()
        for param in optimizer.param_groups:
            param['lr']=lr
        
        start_time=time.time()

        optimizer.zero_grad()
        losses=model([batch_imgs,batch_boxes,batch_classes])
        loss=losses[-1]
        loss.backward()
        optimizer.step()

        end_time=time.time()
        cost_time=int((end_time-start_time)*1000)

        print("global_steps:%d epoch:%d steps:%d/%d cls_loss:%.4f cnt_loss:%.4f reg_loss:%.4f cost_time:%dms lr=%.4e"%\
            (GLOBAL_STEPS,epoch+1,epoch_step+1,steps_per_epoch,losses[0],losses[1],losses[2],cost_time,lr))
        
        writer.add_scalar("loss/cls_loss",losses[0],global_step=GLOBAL_STEPS)
        writer.add_scalar("loss/cnt_loss",losses[1],global_step=GLOBAL_STEPS)
        writer.add_scalar("loss/reg_loss",losses[2],global_step=GLOBAL_STEPS)
        writer.add_scalar("lr",lr,global_step=GLOBAL_STEPS)

        GLOBAL_STEPS+=1
    
    torch.save(model.state_dict(),"/kaggle/working/voc2012_512x800_epoch%d_loss%.4f.pth"%(epoch+1,loss.item()))

# inference

In [ ]:
%cd /kaggle/input/fcosss/FCOS
!ls

In [ ]:
import cv2
import torch
import numpy as np
from torchvision import transforms
from model.fcos import FCOSDetector
import matplotlib.pyplot as plt
def preprocess_img(image,input_ksize):
    '''
    resize image and bboxes 
    Returns
    image_paded: input_ksize
    '''
    min_side, max_side    = input_ksize
    h,  w, _  = image.shape

    smallest_side = min(w,h)
    largest_side=max(w,h)
    scale=min_side/smallest_side
    if largest_side*scale>max_side:
        scale=max_side/largest_side
    nw, nh  = int(scale * w), int(scale * h)
    image_resized = cv2.resize(image, (nw, nh))

    pad_w=32-nw%32
    pad_h=32-nh%32

    image_paded = np.zeros(shape=[nh+pad_h, nw+pad_w, 3],dtype=np.uint8)
    image_paded[:nh, :nw, :] = image_resized
    return image_paded

model=FCOSDetector(mode="inference")
# %cd ../..
model.load_state_dict(torch.load("/kaggle/input/nn-hw2-inference/voc2012_512x800_epoch7_loss0.8541.pth",
                                 map_location=torch.device('cpu')))
model=model.cuda().eval()


In [ ]:
scores[0]

In [ ]:
img = cv2.cvtColor(cv2.imread(f'/kaggle/input/nn-hw2-inference/cat.jpg'),cv2.COLOR_BGR2RGB)
img.shape

In [ ]:
import matplotlib.pyplot as plt
def rescale(box,h,w):
    min_side, max_side = [512,800]
    smallest_side = min(w,h)
    largest_side=max(w,h)
    scale=min_side/smallest_side
    if largest_side*scale>max_side:
        scale=max_side/largest_side
    box[:, [0, 2]] = box[:, [0, 2]] / scale 
    box[:, [1, 3]] = box[:, [1, 3]] / scale
    return box
def draw(s):
    plt.figure()
    c_d = {'person':'blue','cat':'red','dog':'brown'}
    img = cv2.cvtColor(cv2.imread(f'/kaggle/input/nn-hw2-inference/{s}.jpg'),cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    h,w = img.shape[0],img.shape[1]
    img = preprocess_img(img,[512,800])
    img = transforms.ToTensor()(img).unsqueeze(0)
    with torch.no_grad():
        out=model(img.cuda())
    scores = out[0][0]
    boxes = out[2][0]
    print(boxes.shape)
    boxes = rescale(boxes,h,w)
    print(boxes)

    ax = plt.gca()
    for i in range(len(scores)):
        if scores[i].item()>0.5:
            box = boxes[i]
            ax.add_patch(plt.Rectangle((box[0].item(), box[1].item()), 
                                       (box[2]-box[0]).item(), (box[3]-box[1]).item(), color=f"{c_d[s]}", fill=False, linewidth=1))
            ax.text(box[0].item(), box[1].item(), f"{s} {round(scores[i].item(),2)}", bbox={'facecolor':f"{c_d[s]}", 'alpha':0.5})
    plt.savefig(f"/kaggle/working/{s}_fcos.jpg")
draw('person')
draw('cat')
draw('dog')